In [101]:
import os 
import tarfile
import urllib

DOWNLOAD_ROOT = "https://raw.githubusercontent.com/ageron/handson-ml2/master/"
HOUSING_PATH = os.path.join("datasets", "housing")
HOUSING_URL = DOWNLOAD_ROOT +  "datasets/housing/housing.tgz"

# a function to fetch the data
def fetch_housing_data(housing_url=HOUSING_URL, housing_path=HOUSING_PATH):
    os.makedirs(housing_path, exist_ok=True)
    tgz_path = os.path.join(housing_path, "housing.tgz")
    urllib.request.urlretrieve(housing_url, tgz_path)
    housing_tgz = tarfile.open(tgz_path)
    housing_tgz.extractall(path=housing_path)
    housing_tgz.close()

In [80]:
import pandas as pd

# Loading Housing Data
def load_housing_data(housing_path=HOUSING_PATH):
    csv_path = os.path.join(housing_path, "housing.csv")
    return pd.read_csv(csv_path)

In [104]:
housing = load_housing_data()

# general info about the data
# general_info = housing.info()


# getting first few elements
head = housing.head()
# print(head)


# print(housing["ocean_proximity"].value_counts())

# summary of each numerical attribute
# housing.describe()

In [107]:
%matplotlib inline
import matplotlib.pyplot as plt


# we can plot a histogram for each numerical attribute
# housing.hist(bins=50, figsize=(20,15))
# plt.show()

In [116]:
import numpy as np

# creating a test set
# def split_train_test(data, test_ratio):
#     shuffled_indices = np.random.permutation(len(data))
#     test_set_size = int(len(data) * test_ratio)
#     test_indices = shuffled_indices[: test_set_size]
#     train_indices = shuffled_indices[test_set_size:]
#     return data.iloc[train_indices], data.iloc[test_indices]


# train_set, test_set = split_train_test(housing, 0.2)

# print(f"Train Set shape: {len(train_set)}") #16512
# print(f"Test Set shape: {len(test_set)}") #4128

In [119]:
# from sklearn.model_selection import train_test_split
# train_set, test_set = train_test_split(housing, test_size = 0.2, random_state=42)

print(f"Train Set shape: {len(train_set)}") #16512
print(f"Test Set shape: {len(test_set)}") #4128

Train Set shape: 16512
Test Set shape: 4128
